In [1]:
from tqdm import tqdm
from glob import glob
import os
import random
import av
import numpy as np
import matplotlib.pyplot as plt
import pickle
import multiprocessing
from joblib import Parallel, delayed
from insightface.app.face_analysis import FaceAnalysis
# https://github.com/deepinsight/insightface
# THIS REPO IS AMAZING!!
# Face = collections.namedtuple('Face', [
#     'bbox', 'landmark', 'det_score', 'embedding', 'gender', 'age',
#     'embedding_norm', 'normed_embedding'
# ])

def video2numpy(path):
    container = av.open(path)
    frames = {}
    for idx, frame in enumerate(container.decode(video=0)):
        numpy_RGB = np.array(frame.to_image())
        frames[idx] = numpy_RGB
    container.close()

    return frames

def create_dirs(all_vids_paths):
    for bar in set([os.path.dirname(foo) for foo in all_vids_paths]):        
        os.makedirs(bar.replace('raw-videos', 'faces'), exist_ok=True)

def get_paths():
    all_vids_paths = glob(f'../*/raw-videos/*/*')
    create_dirs(all_vids_paths)
    random.shuffle(all_vids_paths)

    return all_vids_paths

def batch_paths(all_vids_paths, n_jobs):
    batch_size = len(all_vids_paths) // n_jobs
    batched = [all_vids_paths[i*batch_size: (i+1)*batch_size] for i in range(n_jobs)]

    batched[-1] += all_vids_paths[batch_size*n_jobs:]

    assert sorted(all_vids_paths) == sorted([bar for foo in batched for bar in foo])


    return batched

def process_paths(all_vids_path, gpu_id):

    fa = FaceAnalysis(det_name='retinaface_r50_v1',
                    rec_name='arcface_r100_v1',
                    ga_name='genderage_v1')

    fa.prepare(ctx_id=gpu_id)
    for videopath in tqdm(all_vids_path):
        frames = video2numpy(videopath)
        if '.mp4' in videopath:
            savepath = videopath.replace('raw-videos', 'faces').replace('.mp4', '.pkl')
        elif '.avi' in videopath:
            savepath = videopath.replace('raw-videos', 'faces').replace('.avi', '.pkl')
        else:
            raise FileNotFoundError(f"{videopath} not a legit video")

        detections = {}
        for idx, frame in frames.items():
            results = fa.get(frame)
            detections[idx] = results
        
        pickle.dump(detections, open(savepath, 'wb'))




In [2]:
NUM_CORES = multiprocessing.cpu_count()
n_jobs = NUM_CORES // 2
gpu_id = -1
batched = batch_paths(get_paths(), n_jobs)

Parallel(n_jobs=n_jobs)(delayed(process_paths)(batch, gpu_id)
                   for batch in tqdm(batched))



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/tk/.virtualenvs/dev-python-3.7/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py", line 567, in run
    self.flag_executor_shutting_down()
  File "/home/tk/.virtualenvs/dev-python-3.7/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py", line 756, in flag_executor_shutting_down
    self.kill_workers()
  File "/home/tk/.virtualenvs/dev-python-3.7/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py", line 766, in kill_workers
    recursive_terminate(p)
  File "/home/tk/.virtualenvs/dev-python-3.7/lib/python3.7/site-packages/joblib/externals/loky/backend/utils.p

TypeError: object of type 'NoneType' has no len()